In [5]:
import pandas as pd
from pyspark.sql import *
from pyspark.sql.functions import *
import sys
sys.path.append('../taxonomy/')
from taxonomy import Taxonomy
from tqdm import tqdm
tqdm.pandas()

In [7]:
taxonomy = Taxonomy(version='v1.1')

AttributeError: 'list' object has no attribute 'categories'

In [2]:
FILES_PATH = '/scratch/WikipediaImagesTaxonomy/commonswiki-20220220-files.parquet'
CATEGORIES_PATH = '/scratch/WikipediaImagesTaxonomy/commonswiki-20220220-category-network.parquet'
LGRAPH_PATH = '/scratch/WikipediaImagesTaxonomy/20220220-category-graph-wlabels_heuristics_simple_v1.0.pkl.bz2'
LGRAPH_H_PATH = '/scratch/WikipediaImagesTaxonomy/20220220-clean-graph-wlabels_heuristics_simple_v1.0.pkl.bz2'

In [3]:
files = pd.read_parquet(FILES_PATH)

In [4]:
print(files.shape)
files.head(5)

(3891446, 4)


,id,title,url,categories
0,80902489,"""A Gathering of Court Women"", Folio from the D...",c/c1/%22A_Gathering_of_Court_Women%22%2C_Folio...,"[Department of Islamic Art, Metropolitan Museu..."
1,53631642,"""Akdeniz"" - Aliağa, 2016.jpg",f/f8/%22Akdeniz%22_-_Alia%C4%9Fa%2C_2016.jpg,"[Akdeniz (ship, 1955), Aliağa ship-breaking yard]"
2,74785910,"""Carnevale di Torino"". Felice Cerruti - f.lli ...",1/13/%22Carnevale_di_Torino%22._Felice_Cerruti...,"[Felice Cerruti Bauduc, Victor Emmanuel II of ..."
3,14890941,"""Escena de playa con figura"".jpg",c/c2/%22Escena_de_playa_con_figura%22.jpg,"[Pedro Lira, People with dogs in art]"
4,17828296,"""Florero con plátanos, limones y libros"", Juan...",d/da/%22Florero_con_pl%C3%A1tanos%2C_limones_y...,"[Works by Juan de Echevarría, Still-life paint..."


In [5]:
categories = pd.read_parquet(CATEGORIES_PATH)

In [6]:
categories.head(5)

,id,title,parents,hiddencat,childs
0,51586720,"'s-Gravelandse Veer 9, Amsterdam","['s-Gravelandseveer, Amsterdam, Rijksmonumente...",False,None
1,76297768,"'s-Gravenhekje 2, Amsterdam","[Rijksmonumenten in Amsterdam-Centrum, 's-Grav...",False,None
2,11456822,'t Oude Huis Zoetermeer,"[Buildings in Zoetermeer, Rijksmonumenten in Z...",False,None
3,75625387,(315898) 2008 QD4,[Centaurs (minor planets)],False,None
4,106082675,(an) Schloßstraße 17 (Ruppertsberg),"[Cultural heritage monuments in Ruppertsberg, ...",False,None


In [7]:
taxonomy = Taxonomy()
taxonomy.load_graph(LGRAPH_PATH)
taxonomy.load_graph(LGRAPH_H_PATH, clean=True)

In [8]:
def queryFile(file, how='heuristics_simple'):
    '''
    Given one file, a row of the files DataFrame, queries recursively all
    the categories and returns the final labels.
    '''

    labels = set()
    for category in file.categories:
        cat_labels = taxonomy.get_label(category, how=how)
        labels |= cat_labels
    return labels

In [9]:
files['labels'] = files.progress_apply(lambda x: queryFile(x), axis=1)

100%|██████████| 3891446/3891446 [01:50<00:00, 35280.43it/s]


In [10]:
files.to_json('data/commonswiki-20220914-files-heuristic-labels.json.bz2')

In [12]:
print(files.shape)
files.head()

(3891446, 5)


,id,title,url,categories,labels
0,80902489,"""A Gathering of Court Women"", Folio from the D...",c/c1/%22A_Gathering_of_Court_Women%22%2C_Folio...,"[Department of Islamic Art, Metropolitan Museu...","{Art, Culture}"
1,53631642,"""Akdeniz"" - Aliağa, 2016.jpg",f/f8/%22Akdeniz%22_-_Alia%C4%9Fa%2C_2016.jpg,"[Akdeniz (ship, 1955), Aliağa ship-breaking yard]",{}
2,74785910,"""Carnevale di Torino"". Felice Cerruti - f.lli ...",1/13/%22Carnevale_di_Torino%22._Felice_Cerruti...,"[Felice Cerruti Bauduc, Victor Emmanuel II of ...","{Places, Art, People, Culture}"
3,14890941,"""Escena de playa con figura"".jpg",c/c2/%22Escena_de_playa_con_figura%22.jpg,"[Pedro Lira, People with dogs in art]",{People}
4,17828296,"""Florero con plátanos, limones y libros"", Juan...",d/da/%22Florero_con_pl%C3%A1tanos%2C_limones_y...,"[Works by Juan de Echevarría, Still-life paint...","{Art, Culture}"
